In [1]:
def print_args(name, *args):  # <1>
    cls_name = args[0].__class__.__name__
    arg_names = ['self', 'instance', 'owner']
    if name == 'set':
        arg_names[-1] = 'value'
    print('{}.__{}__() invoked with args:'.format(cls_name, name))
    for arg_name, value in zip(arg_names, args):
        print('    {:8} = {}'.format(arg_name, value))

In [2]:
class Overriding:  # <2>
    """a.k.a. data descriptor or enforced descriptor"""

    def __get__(self, instance, owner):
        print_args('get', self, instance, owner)  # <3>

    def __set__(self, instance, value):
        print_args('set', self, instance, value)

In [3]:
class OverridingNoGet:  # <4>
    """an overriding descriptor without ``__get__``"""

    def __set__(self, instance, value):
        print_args('set', self, instance, value)

In [4]:
class NonOverriding:  # <5>
    """a.k.a. non-data or shadowable descriptor"""

    def __get__(self, instance, owner):
        print_args('get', self, instance, owner)

In [5]:
class Model:  # <6>
    over = Overriding()
    over_no_get = OverridingNoGet()
    non_over = NonOverriding()

    def spam(self):  # <7>
        print('Model.spam() invoked with arg:')
        print('    self =', self)

#### Overriding descriptor (a.k.a. data descriptor or enforced descriptor):

In [6]:
>>> obj = Model()
>>> obj.over  # doctest: +ELLIPSIS

Overriding.__get__() invoked with args:
    self     = <__main__.Overriding object at 0x0000028A90F81D80>
    instance = <__main__.Model object at 0x0000028A90F82800>
    owner    = <class '__main__.Model'>


In [7]:
>>> Model.over  # doctest: +ELLIPSIS

Overriding.__get__() invoked with args:
    self     = <__main__.Overriding object at 0x0000028A90F81D80>
    instance = None
    owner    = <class '__main__.Model'>


#### An overriding descriptor cannot be shadowed by assigning to an instance:

In [8]:
>>> obj = Model()
>>> obj.over = 7  # doctest: +ELLIPSIS

Overriding.__set__() invoked with args:
    self     = <__main__.Overriding object at 0x0000028A90F81D80>
    instance = <__main__.Model object at 0x0000028A90F83AF0>
    value    = 7


In [9]:
>>> obj.over  # doctest: +ELLIPSIS

Overriding.__get__() invoked with args:
    self     = <__main__.Overriding object at 0x0000028A90F81D80>
    instance = <__main__.Model object at 0x0000028A90F83AF0>
    owner    = <class '__main__.Model'>


#### Not even by poking the attribute into the instance ``__dict__``:

In [10]:
>>> obj.__dict__['over'] = 8
>>> obj.over  # doctest: +ELLIPSIS

Overriding.__get__() invoked with args:
    self     = <__main__.Overriding object at 0x0000028A90F81D80>
    instance = <__main__.Model object at 0x0000028A90F83AF0>
    owner    = <class '__main__.Model'>


In [11]:
>>> vars(obj)

{'over': 8}

#### Overriding descriptor without ``__get__``:

In [12]:
>>> obj.over_no_get  # doctest: +ELLIPSIS

In [13]:
>>> Model.over_no_get   # doctest: +ELLIPSIS

In [14]:
>>> obj.over_no_get = 7  # doctest: +ELLIPSIS

OverridingNoGet.__set__() invoked with args:
    self     = <__main__.OverridingNoGet object at 0x0000028A90F829B0>
    instance = <__main__.Model object at 0x0000028A90F83AF0>
    value    = 7


In [15]:
>>> obj.over_no_get  # doctest: +ELLIPSIS

#### Poking the attribute into the instance ``__dict__`` means you can read the new value for the attribute, but setting it still triggers ``__set__``:

In [16]:
>>> obj.__dict__['over_no_get'] = 9
>>> obj.over_no_get

9

In [17]:
>>> obj.over_no_get = 7  # doctest: +ELLIPSIS

OverridingNoGet.__set__() invoked with args:
    self     = <__main__.OverridingNoGet object at 0x0000028A90F829B0>
    instance = <__main__.Model object at 0x0000028A90F83AF0>
    value    = 7


In [18]:
>>> obj.over_no_get

9

#### Non-overriding descriptor (a.k.a. non-data descriptor or shadowable descriptor):

In [19]:
>>> obj = Model()
>>> obj.non_over  # doctest: +ELLIPSIS

NonOverriding.__get__() invoked with args:
    self     = <__main__.NonOverriding object at 0x0000028A90F829E0>
    instance = <__main__.Model object at 0x0000028A90F830D0>
    owner    = <class '__main__.Model'>


In [20]:
>>> Model.non_over  # doctest: +ELLIPSIS

NonOverriding.__get__() invoked with args:
    self     = <__main__.NonOverriding object at 0x0000028A90F829E0>
    instance = None
    owner    = <class '__main__.Model'>


#### A non-overriding descriptor can be shadowed by assigning to an instance:

In [21]:
>>> obj.non_over = 7
>>> obj.non_over

7

#### Methods are non-over descriptors:

In [22]:
>>> obj.spam  # doctest: +ELLIPSIS

<bound method Model.spam of <__main__.Model object at 0x0000028A90F830D0>>

In [23]:
>>> Model.spam  # doctest: +ELLIPSIS

<function __main__.Model.spam(self)>

In [24]:
>>> obj.spam()  # doctest: +ELLIPSIS

Model.spam() invoked with arg:
    self = <__main__.Model object at 0x0000028A90F830D0>


In [25]:
>>> obj.spam = 7
>>> obj.spam

7

#### No descriptor type survives being overwritten on the class itself:

In [26]:
>>> Model.over = 1
>>> obj.over

1

In [27]:
>>> Model.over_no_get = 2
>>> obj.over_no_get

2

In [28]:
>>> Model.non_over = 3
>>> obj.non_over

7